In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split


# Scaling
from sklearn.preprocessing import StandardScaler, MinMaxScaler, PowerTransformer, MaxAbsScaler
from sklearn.pipeline import Pipeline

import datetime as dt
from time import time

from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier, ExtraTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.svm import SVC
from sklearn.ensemble import BaggingClassifier
from sklearn.naive_bayes import GaussianNB
from lightgbm import LGBMClassifier
from sklearn.neural_network import MLPClassifier

from sklearn.ensemble import GradientBoostingClassifier, AdaBoostClassifier
from sklearn.linear_model import SGDClassifier

from sklearn.metrics import make_scorer, accuracy_score, f1_score
from sklearn.model_selection import cross_val_score

from sklearn.preprocessing import LabelEncoder
import category_encoders as ce

from IPython.display import display


pd.set_option('display.max_columns', None)

In [2]:
df_train = pd.read_csv('Train.csv')
df_test = pd.read_csv('Test.csv')

### **Combine Train & Test dataset for preprocessing**

In [3]:
display([df_train.shape,df_test.shape])
training_data = df_train.shape[0]

[(12079, 14), (1202, 13)]

In [4]:
df_combined = pd.concat([df_train, df_test]).reset_index(drop=True)
display(df_combined.head())

,ID,Policy Start Date,Policy End Date,Gender,Age,First Transaction Date,No_Pol,Car_Category,Subject_Car_Colour,Subject_Car_Make,LGA_Name,State,ProductName,target
0,ID_0040R73,2010-05-14,2011-05-13,Male,30,2010-05-14,1,Saloon,Black,TOYOTA,NaN,NaN,Car Classic,0.0
1,ID_0046BNK,2010-11-29,2011-11-28,Female,79,2010-11-29,1,JEEP,Grey,TOYOTA,NaN,NaN,Car Classic,1.0
2,ID_005QMC3,2010-03-21,2011-03-20,Male,43,2010-03-21,1,Saloon,Red,TOYOTA,NaN,NaN,Car Classic,0.0
3,ID_0079OHW,2010-08-21,2011-08-20,Male,2,2010-08-21,1,NaN,NaN,NaN,NaN,NaN,CarSafe,0.0
4,ID_00BRP63,2010-08-29,2010-12-31,Entity,20,2010-08-29,3,NaN,NaN,NaN,Lagos,Lagos,Muuve,1.0


### **Data Preprocessing**

**Convert to appropriate data types**

In [5]:
date_cols = [col for col in df_combined.columns if "Date" in col]
display(date_cols)

for col in date_cols:
    df_combined[col] = pd.to_datetime(df_combined[col])

['Policy Start Date', 'Policy End Date', 'First Transaction Date']

In [6]:
cat_columns = ['Gender', 'Car_Category', 'Subject_Car_Colour', 'LGA_Name', 'State', 'ProductName']

for col in cat_columns:
    df_combined[col] = df_combined[col].astype('str')

In [7]:
df_combined['company_type'] = df_combined[df_combined['Gender'] == 'Entity']['Gender']

In [8]:
display(df_combined['Gender'].value_counts())

mapper = {
    'Entity': np.nan,
    'Joint Gender': 'Other',
    'NOT STATED': 'Other',
    'NO GENDER': 'Other',
    'SEX': 'Other'
}

df_combined['Gender'] = df_combined['Gender'].replace(mapper)
display(df_combined['Gender'].value_counts())

Male            8356
Female          3679
nan              400
Entity           300
Joint Gender     238
NOT STATED       190
NO GENDER         76
SEX               42
Name: Gender, dtype: int64

Male      8356
Female    3679
Other      546
nan        400
Name: Gender, dtype: int64

In [9]:
string_cols = df_combined.select_dtypes(['category', 'object']).columns.difference(['ID'])
display(string_cols)

from collections import OrderedDict
import re

for col in string_cols:
    df_combined[col] = df_combined[col].apply(lambda x: re.sub("['!@#$%*><^\]\[()=_+{}:\";?,.\/-]+", ' ', x) if isinstance(x, str) else x)

# remove name repetitions withn category
for col in string_cols:
    df_combined[col] = df_combined[col].str.split().apply(lambda x: OrderedDict.fromkeys(x).keys() if isinstance(x, str) else x).str.join(" ").str.title().str.strip()

Index(['Car_Category', 'Gender', 'LGA_Name', 'ProductName', 'State',
       'Subject_Car_Colour', 'Subject_Car_Make', 'company_type'],
      dtype='object')

**Unique value for categorical features**

In [10]:
for col in string_cols:
    print(col+ ' unique values')
    display(df_combined[col].value_counts())
    print('*'*50)
    
# replace with "Other" where value is 1 for each cateogory
for col in string_cols:
    counts = df_combined[col].value_counts()
    idx = counts[counts == 1].index
    df_combined.loc[df_combined[col].isin(idx), col] = 'Other'

Car_Category unique values


Saloon                     6633
Nan                        4110
Jeep                       2223
Truck                       108
Bus                          56
Mini Bus                     45
Pick Up                      32
Motorcycle                   18
Sedan                        14
Mini Van                     13
Wagon                        10
Shape Of Vehicle Chasis       6
Station 4 Wheel               6
Van                           3
Pick Up 3 Tons                2
Tipper Truck                  1
Camry Car Hire                1
Name: Car_Category, dtype: int64

**************************************************
Gender unique values


Male      8356
Female    3679
Other      546
Nan        400
Name: Gender, dtype: int64

**************************************************
LGA_Name unique values


...

**************************************************
Subject_Car_Make unique values


Toyota        5466
Honda         1146
Lexus          654
Mercedes       577
Hyundai        512
              ... 
Tata             1
Howo             1
Changan          1
Rols Royce       1
Commander        1
Name: Subject_Car_Make, Length: 74, dtype: int64

**************************************************
company_type unique values


Entity    300
Name: company_type, dtype: int64

**************************************************


### **Feature Engineering**

**Get date features**

In [11]:
display(date_cols)
date_features = [
    'month', 'day', 'dayofyear', 'dayofweek',
    'days_in_month', 'quarter', 'is_month_start', 'is_month_end', 
    'is_quarter_start', 'is_quarter_end', 'is_year_start', 'is_year_end', 
    'is_leap_year'
]

for col in date_cols:
    for feature in date_features:
        df_combined[col+'_'+feature] = getattr(df_combined[col].dt, feature)
        
for col in date_cols:
    for feature in ['week']:
        df_combined[col+'_'+feature] = getattr(df_combined[col].dt.isocalendar(), feature)
        
# weekend?
for col in date_cols:
    df_combined[col+'_isweekend'] = np.where(df_combined[col].dt.day_name().isin(['Saturday', 'Sunday']), True, False)

['Policy Start Date', 'Policy End Date', 'First Transaction Date']

In [12]:
is_cols = [col for col in df_combined.columns if '_is' in col]
display(is_cols)

for col in is_cols:
    print(col+ ': unique features')
    display(df_combined[col].value_counts())
    print('*'*50)

['Policy Start Date_is_month_start',
 'Policy Start Date_is_month_end',
 'Policy Start Date_is_quarter_start',
 'Policy Start Date_is_quarter_end',
 'Policy Start Date_is_year_start',
 'Policy Start Date_is_year_end',
 'Policy Start Date_is_leap_year',
 'Policy End Date_is_month_start',
 'Policy End Date_is_month_end',
 'Policy End Date_is_quarter_start',
 'Policy End Date_is_quarter_end',
 'Policy End Date_is_year_start',
 'Policy End Date_is_year_end',
 'Policy End Date_is_leap_year',
 'First Transaction Date_is_month_start',
 'First Transaction Date_is_month_end',
 'First Transaction Date_is_quarter_start',
 'First Transaction Date_is_quarter_end',
 'First Transaction Date_is_year_start',
 'First Transaction Date_is_year_end',
 'First Transaction Date_is_leap_year',
 'Policy Start Date_isweekend',
 'Policy End Date_isweekend',
 'First Transaction Date_isweekend']

Policy Start Date_is_month_start: unique features


False    12851
True       430
Name: Policy Start Date_is_month_start, dtype: int64

**************************************************
Policy Start Date_is_month_end: unique features


False    12814
True       467
Name: Policy Start Date_is_month_end, dtype: int64

...

**************************************************
Policy End Date_isweekend: unique features


False    8334
True     4947
Name: Policy End Date_isweekend, dtype: int64

**************************************************
First Transaction Date_isweekend: unique features


False    8181
True     5100
Name: First Transaction Date_isweekend, dtype: int64

**************************************************


In [13]:
# drop columns that have only one unique value
df_combined.drop(['First Transaction Date_is_leap_year', 
                  'Policy Start Date_is_leap_year'], axis=1, inplace=True)

In [14]:
df_combined

,ID,Policy Start Date,Policy End Date,Gender,Age,First Transaction Date,No_Pol,Car_Category,Subject_Car_Colour,Subject_Car_Make,LGA_Name,State,ProductName,target,company_type,Policy Start Date_month,Policy Start Date_day,Policy Start Date_dayofyear,Policy Start Date_dayofweek,Policy Start Date_days_in_month,Policy Start Date_quarter,Policy Start Date_is_month_start,Policy Start Date_is_month_end,Policy Start Date_is_quarter_start,Policy Start Date_is_quarter_end,Policy Start Date_is_year_start,Policy Start Date_is_year_end,Policy End Date_month,Policy End Date_day,Policy End Date_dayofyear,Policy End Date_dayofweek,Policy End Date_days_in_month,Policy End Date_quarter,Policy End Date_is_month_start,Policy End Date_is_month_end,Policy End Date_is_quarter_start,Policy End Date_is_quarter_end,Policy End Date_is_year_start,Policy End Date_is_year_end,Policy End Date_is_leap_year,First Transaction Date_month,First Transaction Date_day,First Transaction Date_dayofyear,First Transaction Date_dayofweek,First Transaction Date_days_in_month,First Transaction Date_quarter,First Transaction Date_is_month_start,First Transaction Date_is_month_end,First Transaction Date_is_quarter_start,First Transaction Date_is_quarter_end,First Transaction Date_is_year_start,First Transaction Date_is_year_end,Policy Start Date_week,Policy End Date_week,First Transaction Date_week,Policy Start Date_isweekend,Policy End Date_isweekend,First Transaction Date_isweekend
0,ID_0040R73,2010-05-14,2011-05-13,Male,30,2010-05-14,1,Saloon,Black,Toyota,Nan,Nan,Car Classic,0.0,NaN,5,14,134,4,31,2,False,False,False,False,False,False,5,13,133,4,31,2,False,False,False,False,False,False,False,5,14,134,4,31,2,False,False,False,False,False,False,19,19,19,False,False,False
1,ID_0046BNK,2010-11-29,2011-11-28,Female,79,2010-11-29,1,Jeep,Grey,Toyota,Nan,Nan,Car Classic,1.0,NaN,11,29,333,0,30,4,False,False,False,False,False,False,11,28,332,0,30,4,False,False,False,False,False,False,False,11,29,333,0,30,4,False,False,False,False,False,False,48,48,48,False,False,False
2,ID_005QMC3,2010-03-21,2011-03-20,Male,43,2010-03-21,1,Saloon,Red,Toyota,Nan,Nan,Car Classic,0.0,NaN,3,21,80,6,31,1,False,False,False,False,False,False,3,20,79,6,31,1,False,False,False,False,False,False,False,3,21,80,6,31,1,False,False,False,False,False,False,11,11,11,True,True,True
3,ID_0079OHW,2010-08-21,2011-08-20,Male,2,2010-08-21,1,Nan,Nan,NaN,Nan,Nan,Carsafe,0.0,NaN,8,21,233,5,31,3,False,False,False,False,False,False,8,20,232,5,31,3,False,False,False,False,False,False,False,8,21,233,5,31,3,False,False,False,False,False,False,33,33,33,True,True,True
4,ID_00BRP63,2010-08-29,2010-12-31,NaN,20,2010-08-29,3,Nan,Nan,NaN,Lagos,Lagos,Muuve,1.0,Entity,8,29,241,6,31,3,False,False,False,False,False,False,12,31,365,4,31,4,False,True,False,True,False,True,False,8,29,241,6,31,3,False,False,False,False,False,False,34,52,34,True,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13276,ID_ZTTHC5X,2010-12-05,2011-12-04,Male,67,2010-12-05,2,Jeep,Black,Toyota,Victoria Island,Lagos,Car Classic,NaN,NaN,12,5,339,6,31,4,False,False,False,False,False,False,12,4,338,6,31,4,False,False,False,False,False,False,False,12,5,339,6,31,4,False,False,False,False,False,False,48,48,48,True,True,True
13277,ID_ZUJAFUP,2010-01-14,2011-01-13,Male,43,2010-01-14,1,Saloon,Silver,Hyundai,Surulere,Lagos,Car Classic,NaN,NaN,1,14,14,3,31,1,False,False,False,False,False,False,1,13,13,3,31,1,False,False,False,False,False,False,False,1,14,14,3,31,1,False,False,False,False,False,False,2,2,2,False,False,False
13278,ID_ZWHCTUM,2010-07-26,2011-07-25,Male,30,2010-07-26,1,Truck,White,Iveco,Victoria Island,Lagos,Cvtp,NaN,NaN,7,26,207,0,31,3,False,False,False,False,False,False,7,25,206,0,31,3,False,False,False,False,False,False,False,7,26,207,0,31,3,False,False,False,False,Fal

**Replace Age Outliers with Mean**

In [15]:
def outliers(column):
    Q1, Q3 = np.percentile(column, [25, 75])
    IQR = Q3 - Q1
    lower_range = Q1 - (1.5 * IQR)
    upper_range = Q3 + (1.5 * IQR)
    return lower_range, upper_range
 
outliers(df_combined['Age'])

(12.5, 72.5)

In [16]:
age_mean = df_combined['Age'].mean()
display(age_mean)

df_combined['Age'] = np.where(df_combined['Age'] <= 12.5, age_mean, df_combined['Age'])
df_combined['Age'] = np.where(df_combined['Age'] >= 72.5, age_mean, df_combined['Age']).astype('int64')

42.37557412845418

### **Missing Values**

In [17]:
df_combined.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13281 entries, 0 to 13280
Data columns (total 58 columns):
 #   Column                                   Non-Null Count  Dtype         
---  ------                                   --------------  -----         
 0   ID                                       13281 non-null  object        
 1   Policy Start Date                        13281 non-null  datetime64[ns]
 2   Policy End Date                          13281 non-null  datetime64[ns]
 3   Gender                                   12981 non-null  object        
 4   Age                                      13281 non-null  int64         
 5   First Transaction Date                   13281 non-null  datetime64[ns]
 6   No_Pol                                   13281 non-null  int64         
 7   Car_Category                             13281 non-null  object        
 8   Subject_Car_Colour                       13281 non-null  object        
 9   Subject_Car_Make                       

In [18]:
# fill missing values with out of range number
df_combined = df_combined.replace(np.nan, -9999)

### **Categorical Encoding**

In [19]:
string_cols = df_combined.select_dtypes(['category', 'object']).columns.difference(['ID'])
string_cols

Index(['Car_Category', 'Gender', 'LGA_Name', 'ProductName', 'State',
       'Subject_Car_Colour', 'Subject_Car_Make', 'company_type'],
      dtype='object')

In [20]:
bd = ce.BackwardDifferenceEncoder()
dummies = bd.fit_transform(df_combined[string_cols], df_combined['target'])
display(dummies)

df_combined = df_combined.merge(dummies, left_index=True, right_index=True)
df_combined.drop(string_cols, axis=1, inplace=True)

# drop ID and date cols
df_combined.drop(date_cols, axis=1, inplace=True)
df_combined.drop('ID', axis=1, inplace=True)

C:\Users\oneda\anaconda3\lib\site-packages\category_encoders\utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):


,intercept,Car_Category_0,Car_Category_1,Car_Category_2,Car_Category_3,Car_Category_4,Car_Category_5,Car_Category_6,Car_Category_7,Car_Category_8,Car_Category_9,Car_Category_10,Car_Category_11,Car_Category_12,Car_Category_13,Car_Category_14,Gender_0,Gender_1,Gender_2,Gender_3,LGA_Name_0,LGA_Name_1,LGA_Name_2,LGA_Name_3,LGA_Name_4,LGA_Name_5,LGA_Name_6,LGA_Name_7,LGA_Name_8,LGA_Name_9,LGA_Name_10,LGA_Name_11,LGA_Name_12,LGA_Name_13,LGA_Name_14,LGA_Name_15,LGA_Name_16,LGA_Name_17,LGA_Name_18,LGA_Name_19,LGA_Name_20,LGA_Name_21,LGA_Name_22,LGA_Name_23,LGA_Name_24,LGA_Name_25,LGA_Name_26,LGA_Name_27,LGA_Name_28,LGA_Name_29,LGA_Name_30,LGA_Name_31,LGA_Name_32,LGA_Name_33,LGA_Name_34,LGA_Name_35,LGA_Name_36,LGA_Name_37,LGA_Name_38,LGA_Name_39,LGA_Name_40,LGA_Name_41,LGA_Name_42,LGA_Name_43,LGA_Name_44,LGA_Name_45,LGA_Name_46,LGA_Name_47,LGA_Name_48,LGA_Name_49,LGA_Name_50,LGA_Name_51,LGA_Name_52,LGA_Name_53,LGA_Name_54,LGA_Name_55,LGA_Name_56,LGA_Name_57,LGA_Name_58,LGA_Name_59,LGA_Name_60,LGA_Name_61,LGA_Name_62,LGA_Name_63,LGA_Name_64,LGA_Name_65,LGA_Name_66,LGA_Name_67,LGA_Name_68,LGA_Name_69,LGA_Name_70,LGA_Name_71,LGA_Name_72,LGA_Name_73,LGA_Name_74,LGA_Name_75,LGA_Name_76,LGA_Name_77,LGA_Name_78,LGA_Name_79,LGA_Name_80,LGA_Name_81,LGA_Name_82,LGA_Name_83,LGA_Name_84,LGA_Name_85,LGA_Name_86,LGA_Name_87,LGA_Name_88,LGA_Name_89,LGA_Name_90,LGA_Name_91,LGA_Name_92,LGA_Name_93,LGA_Name_94,LGA_Name_95,LGA_Name_96,LGA_Name_97,LGA_Name_98,LGA_Name_99,LGA_Name_100,LGA_Name_101,LGA_Name_102,LGA_Name_103,LGA_Name_104,LGA_Name_105,LGA_Name_106,LGA_Name_107,LGA_Name_108,LGA_Name_109,LGA_Name_110,LGA_Name_111,LGA_Name_112,LGA_Name_113,LGA_Name_114,LGA_Name_115,LGA_Name_116,LGA_Name_117,LGA_Name_118,LGA_Name_119,LGA_Name_120,LGA_Name_121,LGA_Name_122,LGA_Name_123,LGA_Name_124,LGA_Name_125,LGA_Name_126,LGA_Name_127,LGA_Name_128,LGA_Name_129,LGA_Name_130,LGA_Name_131,LGA_Name_132,LGA_Name_133,LGA_Name_134,LGA_Name_135,LGA_Name_136,LGA_Name_137,LGA_Name_138,LGA_Name_139,LGA_Name_140,LGA_Name_141,LGA_Name_142,LGA_Name_143,LGA_Name_144,LGA_Name_145,LGA_Name_146,LGA_Name_147,LGA_Name_148,LGA_Name_149,LGA_Name_150,LGA_Name_151,LGA_Name_152,LGA_Name_153,LGA_Name_154,LGA_Name_155,LGA_Name_156,LGA_Name_157,LGA_Name_158,LGA_Name_159,LGA_Name_160,LGA_Name_161,LGA_Name_162,LGA_Name_163,LGA_Name_164,LGA_Name_165,LGA_Name_166,LGA_Name_167,LGA_Name_168,LGA_Name_169,LGA_Name_170,LGA_Name_171,LGA_Name_172,LGA_Name_173,LGA_Name_174,LGA_Name_175,LGA_Name_176,LGA_Name_177,LGA_Name_178,LGA_Name_179,LGA_Name_180,LGA_Name_181,LGA_Name_182,LGA_Name_183,LGA_Name_184,LGA_Name_185,LGA_Name_186,LGA_Name_187,LGA_Name_188,LGA_Name_189,LGA_Name_190,LGA_Name_191,LGA_Name_192,ProductName_0,ProductName_1,ProductName_2,ProductName_3,ProductName_4,ProductName_5,ProductName_6,ProductName_7,State_0,State_1,State_2,State_3,State_4,State_5,State_6,State_7,State_8,State_9,State_10,State_11,State_12,State_13,State_14,State_15,State_16,State_17,State_18,State_19,State_20,State_21,State_22,State_23,State_24,State_25,State_26,State_27,State_28,State_29,State_30,State_31,State_32,State_33,State_34,State_35,State_36,State_37,State_38,State_39,State_40,State_41,State_42,State_43,State_44,State_45,State_46,State_47,State_48,State_49,State_50,State_51,State_52,State_53,State_54,State_55,State_56,State_57,State_58,State_59,State_60,State_61,State_62,State_63,State_64,State_65,State_66,State_67,State_68,State_69,State_70,State_71,State_72,State_73,State_74,State_75,State_76,State_77,State_78,State_79,State_80,State_81,State_82,State_83,Subject_Car_Colour_0,Subject_Car_Colour_1,Subject_Car_Colour_2,Subject_Car_Colour_3,Subject_Car_Colour_4,Subject_Car_Colour_5,Subject_Car_Colour_6,Subject_Car_Colour_7,Subject_Car_Colour_8,Subject_Car_Colour_9,Subject_Car_Colour_10,Subject_Car_Colour_11,Subject_Car_Colour_12,Subject_Car_Colour_13,Subject_Car_Colour_14,Subject_Car_Colour_15,Subject_Car_Colour_16,Subject_Car_Colour_17,Subject_Car_Colour_18,Subject_Car_Colour_19,Subject_Car_Colour_20,Subject_Car_Co

In [21]:
# df_combined.drop(['Car_Category_1', 'Gender_1', 'LGA_Name_1', 'ProductName_1', 'State_1', 'Subject_Car_Colour_1', 'Subject_Car_Make_1'], axis=1, inplace=True)

In [22]:
bool_cols = [col for col in df_combined.columns if '_is' in col]
display(bool_cols)

lb = LabelEncoder()
for col in bool_cols:
    df_combined[col] = lb.fit_transform(df_combined[col])

['Policy Start Date_is_month_start',
 'Policy Start Date_is_month_end',
 'Policy Start Date_is_quarter_start',
 'Policy Start Date_is_quarter_end',
 'Policy Start Date_is_year_start',
 'Policy Start Date_is_year_end',
 'Policy End Date_is_month_start',
 'Policy End Date_is_month_end',
 'Policy End Date_is_quarter_start',
 'Policy End Date_is_quarter_end',
 'Policy End Date_is_year_start',
 'Policy End Date_is_year_end',
 'Policy End Date_is_leap_year',
 'First Transaction Date_is_month_start',
 'First Transaction Date_is_month_end',
 'First Transaction Date_is_quarter_start',
 'First Transaction Date_is_quarter_end',
 'First Transaction Date_is_year_start',
 'First Transaction Date_is_year_end',
 'Policy Start Date_isweekend',
 'Policy End Date_isweekend',
 'First Transaction Date_isweekend']

In [23]:
df_combined

,Age,No_Pol,target,Policy Start Date_month,Policy Start Date_day,Policy Start Date_dayofyear,Policy Start Date_dayofweek,Policy Start Date_days_in_month,Policy Start Date_quarter,Policy Start Date_is_month_start,Policy Start Date_is_month_end,Policy Start Date_is_quarter_start,Policy Start Date_is_quarter_end,Policy Start Date_is_year_start,Policy Start Date_is_year_end,Policy End Date_month,Policy End Date_day,Policy End Date_dayofyear,Policy End Date_dayofweek,Policy End Date_days_in_month,Policy End Date_quarter,Policy End Date_is_month_start,Policy End Date_is_month_end,Policy End Date_is_quarter_start,Policy End Date_is_quarter_end,Policy End Date_is_year_start,Policy End Date_is_year_end,Policy End Date_is_leap_year,First Transaction Date_month,First Transaction Date_day,First Transaction Date_dayofyear,First Transaction Date_dayofweek,First Transaction Date_days_in_month,First Transaction Date_quarter,First Transaction Date_is_month_start,First Transaction Date_is_month_end,First Transaction Date_is_quarter_start,First Transaction Date_is_quarter_end,First Transaction Date_is_year_start,First Transaction Date_is_year_end,Policy Start Date_week,Policy End Date_week,First Transaction Date_week,Policy Start Date_isweekend,Policy End Date_isweekend,First Transaction Date_isweekend,intercept,Car_Category_0,Car_Category_1,Car_Category_2,Car_Category_3,Car_Category_4,Car_Category_5,Car_Category_6,Car_Category_7,Car_Category_8,Car_Category_9,Car_Category_10,Car_Category_11,Car_Category_12,Car_Category_13,Car_Category_14,Gender_0,Gender_1,Gender_2,Gender_3,LGA_Name_0,LGA_Name_1,LGA_Name_2,LGA_Name_3,LGA_Name_4,LGA_Name_5,LGA_Name_6,LGA_Name_7,LGA_Name_8,LGA_Name_9,LGA_Name_10,LGA_Name_11,LGA_Name_12,LGA_Name_13,LGA_Name_14,LGA_Name_15,LGA_Name_16,LGA_Name_17,LGA_Name_18,LGA_Name_19,LGA_Name_20,LGA_Name_21,LGA_Name_22,LGA_Name_23,LGA_Name_24,LGA_Name_25,LGA_Name_26,LGA_Name_27,LGA_Name_28,LGA_Name_29,LGA_Name_30,LGA_Name_31,LGA_Name_32,LGA_Name_33,LGA_Name_34,LGA_Name_35,LGA_Name_36,LGA_Name_37,LGA_Name_38,LGA_Name_39,LGA_Name_40,LGA_Name_41,LGA_Name_42,LGA_Name_43,LGA_Name_44,LGA_Name_45,LGA_Name_46,LGA_Name_47,LGA_Name_48,LGA_Name_49,LGA_Name_50,LGA_Name_51,LGA_Name_52,LGA_Name_53,LGA_Name_54,LGA_Name_55,LGA_Name_56,LGA_Name_57,LGA_Name_58,LGA_Name_59,LGA_Name_60,LGA_Name_61,LGA_Name_62,LGA_Name_63,LGA_Name_64,LGA_Name_65,LGA_Name_66,LGA_Name_67,LGA_Name_68,LGA_Name_69,LGA_Name_70,LGA_Name_71,LGA_Name_72,LGA_Name_73,LGA_Name_74,LGA_Name_75,LGA_Name_76,LGA_Name_77,LGA_Name_78,LGA_Name_79,LGA_Name_80,LGA_Name_81,LGA_Name_82,LGA_Name_83,LGA_Name_84,LGA_Name_85,LGA_Name_86,LGA_Name_87,LGA_Name_88,LGA_Name_89,LGA_Name_90,LGA_Name_91,LGA_Name_92,LGA_Name_93,LGA_Name_94,LGA_Name_95,LGA_Name_96,LGA_Name_97,LGA_Name_98,LGA_Name_99,LGA_Name_100,LGA_Name_101,LGA_Name_102,LGA_Name_103,LGA_Name_104,LGA_Name_105,LGA_Name_106,LGA_Name_107,LGA_Name_108,LGA_Name_109,LGA_Name_110,LGA_Name_111,LGA_Name_112,LGA_Name_113,LGA_Name_114,LGA_Name_115,LGA_Name_116,LGA_Name_117,LGA_Name_118,LGA_Name_119,LGA_Name_120,LGA_Name_121,LGA_Name_122,LGA_Name_123,LGA_Name_124,LGA_Name_125,LGA_Name_126,LGA_Name_127,LGA_Name_128,LGA_Name_129,LGA_Name_130,LGA_Name_131,LGA_Name_132,LGA_Name_133,LGA_Name_134,LGA_Name_135,LGA_Name_136,LGA_Name_137,LGA_Name_138,LGA_Name_139,LGA_Name_140,LGA_Name_141,LGA_Name_142,LGA_Name_143,LGA_Name_144,LGA_Name_145,LGA_Name_146,LGA_Name_147,LGA_Name_148,LGA_Name_149,LGA_Name_150,LGA_Name_151,LGA_Name_152,LGA_Name_153,LGA_Name_154,LGA_Name_155,LGA_Name_156,LGA_Name_157,LGA_Name_158,LGA_Name_159,LGA_Name_160,LGA_Name_161,LGA_Name_162,LGA_Name_163,LGA_Name_164,LGA_Name_165,LGA_Name_166,LGA_Name_167,LGA_Name_168,LGA_Name_169,LGA_Name_170,LGA_Name_171,LGA_Name_172,LGA_Name_173,LGA_Name_174,LGA_Name_175,LGA_Name_176,LGA_Name_177,LGA_Name_178,LGA_Name_179,LGA_Name_180,LGA_Name_181,LGA_Name_182,LGA_Name_183,LGA_Name_184,LGA_Name_185,LGA_Name_186,LGA_Name_187,LGA_Name_188,LGA_Name_189,LGA_Name_190,LGA_Name_191,LGA_Name_192,ProductName_0,Produc

In [24]:
df_combined.select_dtypes(['object', 'category']).columns

Index([], dtype='object')

### **Model**

**Separate Training and Testing set**

In [25]:
training_df = df_combined[:training_data]
testing_df = df_combined[training_data:]

main_cols = [col for col in df_combined[training_data:].columns if 'target' not in col] # remove the Target variable from the training set
testing_df = testing_df[main_cols]

# training and testing shapes (Original Format in Rows)
training_df.shape, testing_df.shape

((12079, 437), (1202, 436))

In [26]:
training_df = training_df.drop_duplicates()

In [27]:
X = training_df.iloc[:, training_df.columns != 'target']
y = training_df.iloc[:, training_df.columns == 'target']

**Split data**

In [28]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15, random_state=42)

In [29]:
from imblearn.over_sampling import RandomOverSampler
os = RandomOverSampler()
X_train, y_train = os.fit_resample(X_train, y_train)

In [30]:
pipeline = Pipeline([
    ('std', StandardScaler())
])
X_train = pipeline.fit_transform(X_train)
X_test = pipeline.transform(X_test)

# scale provided test set
testing_df = pipeline.fit_transform(testing_df)

In [31]:
def train_predict(classifier, X_train, y_train, X_test, y_test):
    ''''''
    results = {}
    
    # Training
    start = time()
    clf = classifier.fit(X_train, y_train)
    end = time()
    
    results['train_time'] = end - start
    
    # Testing
    start = time()
    predictions = clf.predict(X_test)
    end = time()
    
    results['test_time'] = end - start
    
    # Accuracy score
#     score = accuracy_score(y_test, predictions)
#     results['accuracy_score'] = score
    
    # f1_score
    f_score = f1_score(y_test, predictions)
    results['f1_score'] = f_score
    
    # cross val-score
#     cross_scores = cross_val_score(clf, X_train, y_train, scoring='f1', cv=10)
#     results['f1_validation_score'] = cross_scores.mean()
    
    print('{} has been trained and tested \U0001F600'.format(classifier.__class__.__name__))
    return results

In [32]:
from catboost import CatBoostClassifier

clf_A = RandomForestClassifier()
# clf_B = SGDClassifier()
# clf_C = LogisticRegression()
clf_D = GradientBoostingClassifier()
# clf_E = DecisionTreeClassifier()
clf_F = XGBClassifier()
# clf_G = KNeighborsClassifier()
clf_H = AdaBoostClassifier()
# clf_J = BaggingClassifier()
# clf_K = GaussianNB()
clf_L = LGBMClassifier()
clf_M = MLPClassifier()
clf_O = CatBoostClassifier()


results = {}
for i, clf in enumerate([clf_A, clf_D, clf_F, clf_H, clf_O]):
    clf_name = clf.__class__.__name__
    results[clf_name] = {}
    results[clf_name][i] = train_predict(clf, X_train, y_train, X_test, y_test)
    
display(results)

<ipython-input-31-e39e5a51bd58>:7: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  clf = classifier.fit(X_train, y_train)


RandomForestClassifier has been trained and tested 😀


C:\Users\oneda\anaconda3\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


GradientBoostingClassifier has been trained and tested 😀


C:\Users\oneda\anaconda3\lib\site-packages\xgboost\sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
C:\Users\oneda\anaconda3\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


...

484:	learn: 0.3524502	total: 13.5s	remaining: 14.4s
485:	learn: 0.3520286	total: 13.6s	remaining: 14.4s
486:	learn: 0.3516671	total: 13.6s	remaining: 14.3s
487:	learn: 0.3514533	total: 13.6s	remaining: 14.3s
488:	learn: 0.3511686	total: 13.6s	remaining: 14.3s
489:	learn: 0.3507596	total: 13.7s	remaining: 14.2s
490:	learn: 0.3505876	total: 13.7s	remaining: 14.2s
491:	learn: 0.3504046	total: 13.7s	remaining: 14.2s
492:	learn: 0.3500807	total: 13.7s	remaining: 14.1s
493:	learn: 0.3499268	total: 13.8s	remaining: 14.1s
494:	learn: 0.3497043	total: 13.8s	remaining: 14.1s
495:	learn: 0.3494530	total: 13.8s	remaining: 14s
496:	learn: 0.3492087	total: 13.8s	remaining: 14s
497:	learn: 0.3490422	total: 13.9s	remaining: 14s
498:	learn: 0.3488691	total: 13.9s	remaining: 14s
499:	learn: 0.3486269	total: 13.9s	remaining: 13.9s
500:	learn: 0.3484503	total: 13.9s	remaining: 13.9s
501:	learn: 0.3482388	total: 14s	remaining: 13.9s
502:	learn: 0.3481422	total: 14s	remaining: 13.8s
503:	learn: 0.3479629	to

645:	learn: 0.3185964	total: 17.7s	remaining: 9.71s
646:	learn: 0.3184859	total: 17.8s	remaining: 9.69s
647:	learn: 0.3182597	total: 17.8s	remaining: 9.66s
648:	learn: 0.3180955	total: 17.8s	remaining: 9.63s
649:	learn: 0.3179398	total: 17.8s	remaining: 9.6s
650:	learn: 0.3177704	total: 17.8s	remaining: 9.57s
651:	learn: 0.3176057	total: 17.9s	remaining: 9.54s
652:	learn: 0.3173513	total: 17.9s	remaining: 9.51s
653:	learn: 0.3170971	total: 17.9s	remaining: 9.48s
654:	learn: 0.3169607	total: 18s	remaining: 9.46s
655:	learn: 0.3168122	total: 18s	remaining: 9.43s
656:	learn: 0.3165440	total: 18s	remaining: 9.4s
657:	learn: 0.3162868	total: 18s	remaining: 9.37s
658:	learn: 0.3161335	total: 18.1s	remaining: 9.35s
659:	learn: 0.3159258	total: 18.1s	remaining: 9.32s
660:	learn: 0.3158087	total: 18.1s	remaining: 9.29s
661:	learn: 0.3156835	total: 18.1s	remaining: 9.27s
662:	learn: 0.3154986	total: 18.2s	remaining: 9.24s
663:	learn: 0.3153366	total: 18.2s	remaining: 9.21s
664:	learn: 0.3151532	

808:	learn: 0.2905297	total: 22.1s	remaining: 5.21s
809:	learn: 0.2904404	total: 22.1s	remaining: 5.19s
810:	learn: 0.2902309	total: 22.1s	remaining: 5.16s
811:	learn: 0.2900853	total: 22.2s	remaining: 5.13s
812:	learn: 0.2900199	total: 22.2s	remaining: 5.1s
813:	learn: 0.2898899	total: 22.2s	remaining: 5.08s
814:	learn: 0.2898137	total: 22.2s	remaining: 5.05s
815:	learn: 0.2896705	total: 22.3s	remaining: 5.02s
816:	learn: 0.2895402	total: 22.3s	remaining: 4.99s
817:	learn: 0.2893051	total: 22.3s	remaining: 4.96s
818:	learn: 0.2891510	total: 22.3s	remaining: 4.94s
819:	learn: 0.2891031	total: 22.4s	remaining: 4.91s
820:	learn: 0.2889181	total: 22.4s	remaining: 4.88s
821:	learn: 0.2887925	total: 22.4s	remaining: 4.85s
822:	learn: 0.2886646	total: 22.4s	remaining: 4.83s
823:	learn: 0.2884613	total: 22.5s	remaining: 4.8s
824:	learn: 0.2882878	total: 22.5s	remaining: 4.77s
825:	learn: 0.2881664	total: 22.5s	remaining: 4.74s
826:	learn: 0.2879663	total: 22.5s	remaining: 4.71s
827:	learn: 0.

967:	learn: 0.2686434	total: 26.2s	remaining: 865ms
968:	learn: 0.2684596	total: 26.2s	remaining: 838ms
969:	learn: 0.2683108	total: 26.2s	remaining: 810ms
970:	learn: 0.2682025	total: 26.2s	remaining: 783ms
971:	learn: 0.2680712	total: 26.3s	remaining: 756ms
972:	learn: 0.2679748	total: 26.3s	remaining: 729ms
973:	learn: 0.2678324	total: 26.3s	remaining: 702ms
974:	learn: 0.2676709	total: 26.3s	remaining: 675ms
975:	learn: 0.2675275	total: 26.3s	remaining: 648ms
976:	learn: 0.2673906	total: 26.4s	remaining: 621ms
977:	learn: 0.2672986	total: 26.4s	remaining: 594ms
978:	learn: 0.2671720	total: 26.4s	remaining: 566ms
979:	learn: 0.2669676	total: 26.4s	remaining: 539ms
980:	learn: 0.2668158	total: 26.5s	remaining: 512ms
981:	learn: 0.2666630	total: 26.5s	remaining: 485ms
982:	learn: 0.2665724	total: 26.5s	remaining: 458ms
983:	learn: 0.2664622	total: 26.5s	remaining: 431ms
984:	learn: 0.2663661	total: 26.5s	remaining: 404ms
985:	learn: 0.2662119	total: 26.6s	remaining: 377ms
986:	learn: 

{'RandomForestClassifier': {0: {'train_time': 11.848515033721924,
   'test_time': 0.08989930152893066,
   'f1_score': 0.1638418079096045}},
 'GradientBoostingClassifier': {1: {'train_time': 27.583609342575073,
   'test_time': 0.023807287216186523,
   'f1_score': 0.3669527896995708}},
 'XGBClassifier': {2: {'train_time': 8.58151912689209,
   'test_time': 0.015637874603271484,
   'f1_score': 0.2900763358778626}},
 'AdaBoostClassifier': {3: {'train_time': 7.954846620559692,
   'test_time': 0.18752598762512207,
   'f1_score': 0.35782747603833864}},
 'CatBoostClassifier': {4: {'train_time': 28.038846492767334,
   'test_time': 0.02394390106201172,
   'f1_score': 0.3559322033898305}}}

In [33]:
from catboost import CatBoostClassifier
params = {
    'loss_function': 'CrossEntropy',# objective function
   # 'loss_function' :'Logloss',
    'iterations': 10000,
   # 'custom_loss': 'F1',
    'learning_rate': 0.02286071,
    'eval_metric':'F1', # metric,
    #'task_type':'GPU',
    #'border_count':32,
    'early_stopping_rounds':200,
    'verbose': 200, # output to stdout info about training process every 200 iterations
   # 'random_seed': SEED
    
}

cat = CatBoostClassifier(**params)
cat.fit(X_train, y_train)

predictions = cat.predict(X_test)
f1_s = f1_score(y_test, predictions)
f1_s

0:	learn: 0.7378925	total: 42.9ms	remaining: 7m 8s
200:	learn: 0.8159402	total: 5.13s	remaining: 4m 9s
400:	learn: 0.8456585	total: 10.4s	remaining: 4m 8s
600:	learn: 0.8716060	total: 16.9s	remaining: 4m 23s
800:	learn: 0.8895054	total: 21.7s	remaining: 4m 9s
1000:	learn: 0.9021359	total: 26.9s	remaining: 4m 1s
1200:	learn: 0.9129090	total: 31.9s	remaining: 3m 53s
1400:	learn: 0.9223635	total: 36.6s	remaining: 3m 44s
1600:	learn: 0.9290422	total: 41.7s	remaining: 3m 38s
1800:	learn: 0.9353646	total: 46.6s	remaining: 3m 32s
2000:	learn: 0.9417820	total: 51.6s	remaining: 3m 26s
2200:	learn: 0.9462175	total: 56.5s	remaining: 3m 20s
2400:	learn: 0.9492600	total: 1m 1s	remaining: 3m 14s
2600:	learn: 0.9517817	total: 1m 6s	remaining: 3m 8s
2800:	learn: 0.9557167	total: 1m 11s	remaining: 3m 2s
3000:	learn: 0.9583473	total: 1m 15s	remaining: 2m 56s
3200:	learn: 0.9605543	total: 1m 20s	remaining: 2m 51s
3400:	learn: 0.9627187	total: 1m 25s	remaining: 2m 45s
3600:	learn: 0.9634024	total: 1m 30s	

0.19487179487179487

In [36]:
subfile = pd.DataFrame(df_test['ID'])
subfile['target'] = cat.predict(testing_df).astype('int64')

In [37]:
subfile['target'].value_counts()

0    1121
1      81
Name: target, dtype: int64

In [125]:
subfile.to_csv('submission1.csv', index=False)